# 项目进度报告（Jupyter Notebook 模板）
**生成时间：** 2025-10-27 07:29  
**作者：** Haidi  

> 用于向 Leader 汇报当前阶段性成果：数据、方法、关键结果、结论与后续计划。


## 1. 项目背景与目标
- **背景**：英语考试“语法填空”智能分析与得分率预测。
- **当前目标（本阶段）**：
  1) 基于题目与知识点画像，使用 LLM Prompt 直接预测各小题得分率；
  2) 完成最小可用流程（加载数据 → 构造 Prompt → 调用接口/离线推断 → 输出报告）；
  3) 产出可视化与文字总结，便于业务/教学同事理解。

> 注：你可以在此处补充本周里程碑、完成度、卡点。


## 2. 可配置参数（先在这里改路径/开关，再运行下面的单元格）
- 将路径替换为你本地/服务器的真实文件位置；Windows 路径请使用 `r"C:\path\to\file.json"` 形式。


In [ ]:
# ====== 配置区（先改这里） ======
from pathlib import Path

# 常用数据路径（按你的实际情况填写）
DATA_DIR = Path(r"C:\work-2025\10\grammar_gap_kp_updated\data")  # ← 修改为你的数据目录
EXAM_JSON = DATA_DIR / "exam.json"
CLUSTER_CSV = DATA_DIR / "cluster_profiles.csv"
KP_JSON = DATA_DIR / "llm_kp_labels.json"            # 小题→知识点（若有答案字段也可放在 exam.json 里）

# 预测输出（可选）
PREDICT_OUT = DATA_DIR / "score_rate_pred.json"

# 是否只演示不联网（True 时跳过真实 LLM 调用）
DRY_RUN = True

print("DATA_DIR =", DATA_DIR)
print("EXAM_JSON exists?", EXAM_JSON.exists())
print("CLUSTER_CSV exists?", CLUSTER_CSV.exists())
print("KP_JSON exists?", KP_JSON.exists())
print("DRY_RUN =", DRY_RUN)


## 3. 数据来源与结构概览
- `exam.json`：段落、空格 id、可选参考答案等；
- `cluster_profiles.csv`：各学生分群后在 5 个技能维度的掌握水平；
- `llm_kp_labels.json`：各小题对应的知识点标签（如：从句运用/时态掌握等）。


In [ ]:
import json, pandas as pd

def safe_load_json(path):
    if not Path(path).exists():
        print(f"[WARN] 文件不存在：{path}")
        return None
    try:
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception as e:
        print(f"[ERROR] 读取 JSON 失败：{e}")
        return None

def safe_load_csv(path):
    if not Path(path).exists():
        print(f"[WARN] 文件不存在：{path}")
        return None
    try:
        return pd.read_csv(path)
    except Exception as e:
        print(f"[ERROR] 读取 CSV 失败：{e}")
        return None

exam = safe_load_json(EXAM_JSON)
clusters = safe_load_csv(CLUSTER_CSV)
kp_map = safe_load_json(KP_JSON)

print("\n=== exam.json 样例 ===")
if exam:
    # 打印前 1 个 block 的前 3 个小题信息
    for block in exam.get("content_list", [])[:1]:
        print("content 预览：", block.get("content","")[:120], "...")
        ts = block.get("topic_score", [])[:3]
        print("topic_score 预览：", ts)
        break

print("\n=== cluster_profiles.csv 概览 ===")
if clusters is not None:
    display_cols = [c for c in clusters.columns[:8]]
    print(clusters.shape)
    display(clusters.head()[display_cols])

print("\n=== llm_kp_labels.json 样例 ===")
if kp_map:
    # 打印前 5 条 id→知识点
    preview = list(kp_map.items())[:5] if isinstance(kp_map, dict) else kp_map[:5]
    print(preview)


## 4. 方法流程（本阶段）
1. **知识点关联**：从 `llm_kp_labels.json`（或由 LLM 判定）拿到每个小题的知识点；
2. **构造 Prompt**：以“题目片段 + 小题 + 知识点 + 人群分布画像”组织提示词；
3. **LLM 推断**：要求模型直接输出 `[0,1]` 区间的得分率（百分比/小数）；
4. **分析与可视化**：生成总览表、直方图/箱线图、Top-K 难易题等；
5. **结论与计划**：总结效果、指出样本外泛化的风险与下一步计划。

> 本阶段不做“LLM 与 baseline 融合”，而是**直接用 LLM 推断**（你之前的 v2 方案的子集）。


## 5. Prompt 草案（可直接复用到你的 `main.py`/API 调用里）
> 你可以把这里的模板复制到项目的 `prompt_templates/score_rate_prompt.txt`。下面单元格会渲染出最终 Prompt。


In [ ]:
PROMPT_TEMPLATE = """
你是一名资深英语命题与解析专家。请基于下列信息，直接给出该小题的**得分率**预测（0~1 之间的小数），并给出20字以内的理由。

【题目片段】
{passage}

【小题 ID】{qid}
【已知知识点（五选一）】{kp}
【分群画像（示例）】
- 共有 {n_clusters} 个学生群体；
- 该知识点在各群体的平均掌握度均值约为 {avg_mastery:.2f}；
- 其中最低群体均值约 {min_mastery:.2f}，最高群体均值约 {max_mastery:.2f}。

【输出格式（JSON 严格遵守）】
{{
  "id": {qid},
  "knowledge_point": "{kp}",
  "score_rate": <0~1 小数>,
  "rationale": "<20字理由>"
}}

只输出 JSON，不要多余文本。
""".strip()

print(PROMPT_TEMPLATE[:400] + " ...")


## 6. 直接 LLM 推断（或 DRY-RUN 模拟）
- 若 `DRY_RUN=True`：生成一个可复现实验的模拟结果（固定随机种子）。
- 正式跑通时，将此处替换为真实的 API 调用函数 `predict_score_rate()`。


In [ ]:
import random, math

import pandas as pd
import matplotlib.pyplot as plt

random.seed(42)

def summarize_mastery(df: pd.DataFrame, kp_col_prefix="Skill_"):
    """
    粗略统计：把含有 kp 名称的列（如 Skill_4_从句运用）取均值/极值，供 Prompt 文案与演示用。
    """
    if df is None or df.empty:
        return 0.5, 0.5, 0.5
    # 简单策略：取所有技能列整体的统计（演示用）
    skill_cols = [c for c in df.columns if c.startswith(kp_col_prefix)]
    if not skill_cols:
        return 0.5, 0.5, 0.5
    row_means = df[skill_cols].mean(axis=1)
    return row_means.mean(), row_means.min(), row_means.max()

avg_m, min_m, max_m = summarize_mastery(clusters if clusters is not None else None)
print(f"Mastery 概览：avg={avg_m:.2f}, min={min_m:.2f}, max={max_m:.2f}")

def dry_run_predict_for_ids(exam_json, kp_map):
    """
    构造 Prompt（展示用），并生成模拟的 score_rate。
    """
    outputs = []
    if not exam_json:
        return outputs
    content_list = exam_json.get("content_list", [])
    for block in content_list:
        passage = block.get("content", "")[:400]  # 截断展示
        for ts in block.get("topic_score", []):
            qid = int(ts["id"])
            kp = None
            if isinstance(kp_map, dict):
                kp = kp_map.get(str(qid), kp_map.get(qid, None))
            elif isinstance(kp_map, list):
                # 若是列表，尝试 { "id": 21, "knowledge_point": "从句运用" } 结构
                for item in kp_map:
                    if int(item.get("id", -1)) == qid:
                        kp = item.get("knowledge_point")
                        break
            kp = kp or "固定搭配与短语"

            prompt = PROMPT_TEMPLATE.format(
                passage=passage, qid=qid, kp=kp,
                n_clusters=int(clusters["Cluster"].nunique()) if clusters is not None and "Cluster" in clusters.columns else 5,
                avg_mastery=avg_m, min_mastery=min_m, max_mastery=max_m
            )
            # 这里你可以把 prompt 落盘用于审阅
            # print(prompt)

            # —— 模拟一个稳定的“得分率” ——
            base = {"从句运用":0.62, "语态转换":0.55, "时态掌握":0.58, "非谓语动词":0.52, "固定搭配与短语":0.68}.get(kp, 0.60)
            jitter = (hash(qid) % 7) * 0.01  # 0~0.06
            score_rate = max(0.05, min(0.95, base + (avg_m-0.5)*0.2 + jitter))

            outputs.append({
                "id": qid,
                "knowledge_point": kp,
                "score_rate": round(float(score_rate), 3),
                "rationale": "依据画像与语境的整体判断"
            })
    return outputs

if DRY_RUN:
    preds = dry_run_predict_for_ids(exam, kp_map)
else:
    preds = []  # ← 在这里替换成真实 predict_score_rate(exam, clusters, kp_map)

import json
print(f"生成结果条数：{len(preds)}")
if preds:
    with open(PREDICT_OUT, "w", encoding="utf-8") as f:
        json.dump(preds, f, ensure_ascii=False, indent=2)
    print("已保存到：", PREDICT_OUT)

# 展示表格
if preds:
    df_pred = pd.DataFrame(preds).sort_values("id")
    display(df_pred.head(10))

    # 简单直方图
    plt.figure()
    plt.hist(df_pred["score_rate"], bins=10)
    plt.title("LLM 得分率分布（模拟）")
    plt.xlabel("score_rate")
    plt.ylabel("count")
    plt.show()
else:
    print("[WARN] 没有生成任何预测结果。请检查数据与配置。")


## 7. 关键结果与解读（示例）
- 分布图：观察整体难易度；
- Top-5 最难/最易小题；
- 分群视角：若需要，可补充按群体的差异（此处演示聚合结果）。


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def topk_easy_hard(df_pred: pd.DataFrame, k=5):
    if df_pred is None or df_pred.empty:
        return None, None
    easy = df_pred.sort_values("score_rate", ascending=False).head(k)
    hard = df_pred.sort_values("score_rate", ascending=True).head(k)
    return easy, hard

if 'df_pred' in globals():
    easy, hard = topk_easy_hard(df_pred, k=5)
    if easy is not None:
        print("\n=== Top-5 最易小题 ===")
        display(easy)
        print("\n=== Top-5 最难小题 ===")
        display(hard)

    # 箱线图
    if 'df_pred' in globals() and not df_pred.empty:
        plt.figure()
        plt.boxplot(df_pred['score_rate'])
        plt.title("得分率箱线图（模拟）")
        plt.ylabel("score_rate")
        plt.show()
else:
    print("[WARN] 还没有 df_pred。先运行上面的单元格。")


## 8. 结论与下一步计划
- **已完成**：数据读取、Prompt 模板、演示级推断与可视化。
- **下一步**：
  1) 接入真实 LLM（或你们的内网大模型）API，落盘每个小题的 Prompt 和原始返回；
  2) 增加**自动校验**（范围/合法 JSON/单位转换），失败样本重试与告警；
  3) 引入评测集（与真实正确率对齐）与误差指标（MAE / MAPE）；
  4) 产出**对业务友好的解释文本**（难易原因、教学建议）。


## 9. 附录
- `main.py` CLI 用法（示例）：
```bash
python main.py --input exam.json --clusters cluster_profiles.csv --kp llm_kp_labels.json --output score_rate_pred.json --dry-run
```
- 目录建议：
```
project/
  ├─ data/
  │   ├─ exam.json
  │   ├─ cluster_profiles.csv
  │   └─ llm_kp_labels.json
  ├─ prompt_templates/
  │   └─ score_rate_prompt.txt
  ├─ src/
  │   └─ main.py
  └─ reports/
      └─ progress.ipynb
```
